In [1]:
from typing import List, Dict, Tuple, Optional
from collections import deque
import heapq

In [ ]:
# 20. Valid Parentheses

class Solution:
    def isValid(self, s: str) -> bool:
        paren = {'(':')', '{':'}', '[':']'}
        store = []
        for p in s:
            if p in paren:
                store.append(p)
            else:
                if not store or p != paren[store.pop()]:
                    return False
        return not store

s = "(())"
a = Solution()
res = a.isValid(s)
res

True

In [2]:
# 48. Rotate Image
class Solution:
    def rotate(self, matrix: List[List[int]]) -> None:
        """
        Do not return anything, modify matrix in-place instead.
        """
        pass

matrix = [[1,2,3],[4,5,6],[7,8,9]]
a = Solution()
res = a.rotate(matrix)
res

In [ ]:
# 215. Kth Largest Element in an Array
'''
- 这道题是要找出数组中第 K 大的元素。输入是一个整数数组 nums 和一个整数 k，输出是数组中第 K 大的数字。
- 使用最简单的解法，可以先对数组进行降序排序，然后取第 K 个元素。时间复杂度是 O(n log n)。
- 我会使用一个优化解法：
    - 使用一个大小为 K 的最小堆，遍历数组，将每个元素加入堆中。
    - 如果堆的大小超过 K，就弹出堆顶（也就是当前最小值）。这样堆中始终保留的是当前最大的 K 个元素，堆顶就是第 K 大的数。
    - 这样做的时间复杂度是 O(n log k)，更高效。
- 总结一下这道题的思路：使用一个大小为 K 的小顶堆维护当前最大的 K 个数，遍历数组过程中更新堆的内容，最终堆顶就是我们要找的第 K 大的元素。
'''
class Solution:
    def findKthLargest(self, nums: List[int], k: int) -> int:
        import heapq
        heap = []
        for n in nums:
            heapq.heappush(heap, n)
            if len(heap) > k:
                heapq.heappop(heap)
        return heap[0]
            
nums = [3,2,1,5,6,4]; k = 2
a = Solution()
res = a.findKthLargest(nums, k)
res

5

In [ ]:
# 347. Top K Frequent Elements
'''
- 这道题是要找出数组中出现频率最高的前 K 个元素。输入是一个整数数组 nums 和一个整数 k，输出是一个包含 k 个元素的列表，这些元素是数组中最常出现的元素。
- 使用最简单的解法，可以先统计每个元素出现的次数，然后将所有元素按频率排序，最后取前 K 个。这样做的时间复杂度是 O(n log n)，其中 n 是数组长度。
- 我会使用一个优化解法：
    - 先用 Counter 统计每个元素出现的频率，然后用一个最小堆（小顶堆）来维护前 K 个高频元素。
    - 每次插入新元素时，如果堆大小超过 K，就弹出堆顶（频率最小的）。最终堆中剩下的就是频率前 K 高的元素。
    - 这样做的时间复杂度是 O(n log k)，适合用于数据量大的场景。
- 总结一下这道题的思路：首先统计频率，然后利用小顶堆动态维护出现频率最高的 K 个元素，最后从堆中提取结果。
'''
class Solution:
    def topKFrequent(self, nums: List[int], k: int) -> List[int]:
        import heapq
        from collections import Counter
        count = Counter(nums)
        heap = []
        for num,freq in count.items():
            heapq.heappush(heap, (freq, num))
            if len(heap) > k:
                heapq.heappop(heap)
        return [ num for _, num in heap]

nums = [1,1,1,2,2,3]; k = 2
a = Solution()
res = a.topKFrequent(nums, k)
res

[2, 1]